# Input data

In [8]:
speed_limit = 40
violation_threshold = 1

# Classic approach

In [9]:
# Dictionary {I: [V, C, T, X, Y]}
# I - object ID
# V - determines whether the objects is a violator
# C - violation time counter
# T - previos timestamp
# X - previous X coordinate
# Y - previos Y coordinate
ids = {}

with open("input/data.csv", 'r', newline='') as csvfile:
    next(csvfile)
    for line in csvfile:
        # Read line
        vals = line.split(',')
        i = vals[1]
        t, x, y = float(vals[0]), float(vals[3]), float(vals[4])

        # Add object if not present
        if i not in ids:
            ids[i] = [False, 0, t, x, y]
            continue
        
        # Skip object if it's a violator
        if ids[i][0]:
            continue

        # Calculate statistics
        dt = t - ids[i][2]
        dx = x - ids[i][3]
        dy = y - ids[i][4]
        s = (dx**2 + dy**2)**0.5
        v = s / dt * 3.6

        # Determine speed limit violation
        if v > speed_limit:
            ids[i][1] += dt
        else:
            ids[i][1] = 0

        # Determine counter exceeding
        if ids[i][1] >= violation_threshold:
            ids[i][0] = True
        
        # Update data
        ids[i][2] = t
        ids[i][3] = x
        ids[i][4] = y

In [10]:
vs = [v[0] for v in ids.values()]
vs_ids = [k for k, v in zip(ids.keys(), vs) if v]
for v in vs_ids:
    print(v)

00000000-0000-0000-0000-000000006508
00000000-0000-0000-0000-000000006629
00000000-0000-0000-0000-000000006467
00000000-0000-0000-0000-000000006647
00000000-0000-0000-0000-000000006605
00000000-0000-0000-0000-000000006660


# Using generators

In [11]:
def trace():
    violation_time = 0
    t_prev, x_prev, y_prev = 0, 0, 0
    while True:
        t, x, y = yield

        if t_prev != 0:
            dt = t - t_prev
            dx = x - x_prev
            dy = y - y_prev

            distance = (dx**2 + dy**2)**0.5
            speed = distance / dt * 3.6
            
            if speed > speed_limit:
                violation_time += dt
            else:
                violation_time = 0

        yield violation_time >= violation_threshold
        t_prev, x_prev, y_prev = t, x, y

In [12]:
def violators(file):
    traces = {}
    with open(file, 'r', newline='') as csvfile:
        next(csvfile)
        for line in csvfile:
            # Read and parse line
            vals = line.split(',')
            id = vals[1]
            t, x, y = float(vals[0]), float(vals[3]), float(vals[4])

            # Add new trace
            if id not in traces:
                tr = trace(); next(tr)
                traces[id] = tr

            tr = traces[id]

            # Skip violators
            if not tr:
                continue
            
            violator = tr.send((t, x, y))
            next(tr)
            
            if violator:
                # Invalidate trace
                traces[id] = False
                yield id

In [13]:
for v in violators("input/data.csv"):
    print(v)

00000000-0000-0000-0000-000000006467
00000000-0000-0000-0000-000000006647
00000000-0000-0000-0000-000000006605
00000000-0000-0000-0000-000000006629
00000000-0000-0000-0000-000000006508
00000000-0000-0000-0000-000000006660


# Assert

In [14]:
set(vs_ids) == set(violators("input/data.csv"))

True